<p> <strong>Nama:</strong> Naufal Prima Yoriko</p>
<p> <strong>Email:</strong> primayoriko@gmail.com </p>
<p> <strong>Username:</strong> primayoriko </p>
<p> <strong>Dicoding Profile:</strong> https://www.dicoding.com/users/primayoriko </p>


# **Submission PaperRockScissors**

Sebelum menuju ke langkah pengerjaan, ini terdapat beberapa referensi yang saya pakai

1.   Neural Network Activation Function in Keras
     *   https://missinglink.ai/guides/neural-network-concepts/7-types-neural-network-activation-functions-right/
     *   https://keras.io/api/layers/activations/

2.   Loss Function in Keras
     *   https://neptune.ai/blog/keras-loss-functions
     *   https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
     *   https://keras.io/api/losses/
     *   https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

3.   Callback in Keras
     * https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/

4.   Optimization in Keras
     * https://keras.io/api/optimizers/ 

5.   Performance comparison
     * https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/
     * https://www.dlology.com/blog/quick-notes-on-how-to-choose-optimizer-in-keras/
     * https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/70253



Berikut ini adalah tahap-tahap pengerjaan yang saya lakukan dalam memeroleh hasil dari Image Processing menggunakan CNN (*Convolutional Neural Network*)

**Pertama**, mendownload file dari web dicoding dengan menggunakan utilitas `wget`

In [1]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-11-01 22:48:04--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  8.25MB/s    in 45s     

2020-11-01 22:48:50 (6.80 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



**Kedua**, melakukan ekstraksi dari file yang didownload dengan menggunakan `zipfile`

In [2]:
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Ketiga**, dengan menggunakan library tambahan yaitu `split-folder`, saya melakukan splitting dari file-file gambar menjadi training (`train`) dan validation (`val`) data. 

**Note:** referensi split saya dapat dari forum dicoding, saya rasa karena dari forum, bukan website luar, tidak apa-apa untuk diikuti

In [3]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/tmp/rockpaperscissors/rps-cv-images', output='/tmp/rockpaperscissors/splitted', seed=1337, ratio=(.6, .4))

Copying files: 2188 files [00:00, 3567.03 files/s]


**Keempat**, saya mencatat direktori dari tiap tipe data, `train` dan `val` data

In [4]:
base_dir = '/tmp/rockpaperscissors/splitted'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
print(train_dir)
print(validation_dir)

/tmp/rockpaperscissors/splitted/train
/tmp/rockpaperscissors/splitted/val


**Kelima**, mencatat direktori dari tiap-tiap kategori gambar dan juga mencatat total file image di tiap segmennya, untuk berjaga semisal akan dibutuhkan kedepannya.

In [5]:
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')
train_rock_dir = os.path.join(train_dir, 'rock')

validation_paper_dir = os.path.join(validation_dir, 'paper')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')
validation_rock_dir = os.path.join(validation_dir, 'rock')

train_paper = len(os.listdir(train_paper_dir))
train_scissors = len(os.listdir(train_scissors_dir))
train_rock = len(os.listdir(train_rock_dir))

validation_paper = len(os.listdir(validation_paper_dir))
validation_scissors = len(os.listdir(validation_scissors_dir))
validation_rock = len(os.listdir(validation_rock_dir))

total_train = train_paper + train_scissors + train_rock
total_validation = validation_paper + validation_scissors + validation_rock

print(total_train)
print(total_validation)

1312
876


**Keenam**, pembuatan generator image dari gambar-gambar yang tersedia, dengan menggunakan `ImageDataGenerator` dari Keras. Fungsi dari image data generator ini adalah untuk membantu mengklasifikasikan suatu gambar ke kategoti tertentu karena pada gambar mungkin ada faktor tertentu yang membedakan tampilan (diluar karena perbedaan objek), seperti jarak/ukuran, orientasi, gradasi warna, dll.

Disini kode yang digunakan kurang lebih seperti yang ada di modul `Latihan Membuat Model Klasifikasi Gambar` di kelas ini.

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    shear_range=0.2,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode = 'nearest'
                  )

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    shear_range=0.2,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode = 'nearest'
                  )

**Keenam**, Membuat pengaturan dimensi image, class, dan juga flow dari transfer data image dalam pembuatan model di tiap batchnya.

Disini kode yang digunakan kurang lebih seperti yang ada di modul `Latihan Membuat Model Klasifikasi Gambar` di kelas ini. Namun, terdapat penyesuaian size tiap batch-nya sekaligus `class_mode` nya.

In [16]:
from math import ceil

# Diasumsikan akan dibuat dibuat 25 step tiap epoch
steps_per_epoch = 25

# Maka berikut ini jumlah file di tiap flow
train_per_batch = ceil(total_train/steps_per_epoch)
val_per_batch = ceil(total_validation/steps_per_epoch)
print(train_per_batch)
print(val_per_batch)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=train_per_batch,
                    class_mode='categorical'
                  )
 
validation_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(150, 150),
                          batch_size=val_per_batch,
                          class_mode='categorical'
                        )

53
36
Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


**Ketujuh**, membuat struktur dari CNN. CNN yang dibuat disini, sesuai spesifikasi menggunakan model sequensial, yang susunannya dibebaskan (asal memiliki > 1 hidden layer).

Disini kode/model yang digunakan memiliki struktur dasar dari `Latihan Membuat Model Klasifikasi Gambar` di kelas ini. Namun, metode penulisan dibuat lebih modular dan juga secara umum dapat dijelaskan bahwa struktur CNN yang sequensial memiliki 1 layer input, 9 layer hidden, dan 1 layer output.

Aksi yang dilakukan antara lain dapat diringkas sebagai berikut


1.   Convolution layerm untuk melakukan konvolusi katriks dari piksel gambar 2D dengan filters digunakan berturut 32, 64, 128, dan 256. Kernel size yang digunakan berukuran 3x3. Activation function adalah menggunakan `relu`, dengan alasan `relu` cukup ringan secara komputasi dan cukup baik dalam mengestimasi hidden layer
2.   MaxPooling Layer untuk melakukan max pool elemen pada matriks dari piksel gambar, yaitu mengambil max value dari tiap submatriks.
3. Dense Layer
4. Flatten Layer untuk melakukan reduksi dimensi dari matriks piksel gambar.
5. Sebelumnya digunakan dropout layer untuk melakukan deaktivasi layer secara random guna menghindari overfitting, namun karena yang ingin dicapai adalah akurasi tertinggi maka layer ini dihilangkan.
6. Output layer guna mengeluarkan hasil akhir, pada layer ini digunakan activation function yang berbeda. Dari explorasi sumber dan experimen langsung, ditemukan activation function yang baik adalah `softmax` dan `sigmoid`, namun karena `softmax` sedikit lebih baik (khusus untuk activation akhir/output karena bisa identifikasi kelas) maka digunakan `softmax`




In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dense(512, activation= 'relu'))
model.add(Flatten())
model.add(Dense(3, activation= 'softmax'))

**Kedelapan**, menggunakan `model.compile()` untuk melakukan setting sebelum melakukan ftting dari model kita, disini digunakan beberapa bonus yang tidak dijelaskan secara mendalam di kelas, yaitu 


1.   Loss function
2.   Optimizer
3.   Metrics



In [ ]:
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

loss = 'categorical_crossentropy'

# Untuk opsi lain dari loss, uncomment salah satu dibawah ini

# loss = SigmoidFocalCrossEntropy()

opt = Adam(amsgrad=True, lr=0.001)

# Untuk opsi lain dari optimizer, uncomment salah satu dibawah ini

# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

# opt = 'rmsprop'

# opt = Adam()

model.compile(loss=loss,
              optimizer=opt,
              metrics=['accuracy'])

**Kesembilan**, 

In [ ]:
from time import time

start_time = time()
schema = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2,
        callbacks=[
           EarlyStopping(patience=4, restore_best_weights=True),
           ReduceLROnPlateau(patience=2)
        ]
      )
print("--- Finished in %s seconds ---" % (time() - start_time))

Dari sini, didapati akurasi sebesar **99.44%** pada `val` data, sedangkan akurasi sebesar **98.02%** pada `train` data sendiri. Untuk waktu eksekusi, training dan validasi data memakan waktu selama **1411 s** ~ **24 min**

**Kesepuluh**, ini opsional, semisal anda ingin mencoba memasukkan untuk menguji suatu image tangan guna diklasifikasikan ke dalam salah satu kategori (paper, rock, scissors).

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# %matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  # imgplot = plt.imshow(img)
  # x = image.img_to_array(img)
  # x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  print(fn)
  print(classes)

  # if classes==0:
  #   print('clean')
  # else:
  #   print('messy')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

Epoch 1/12
25/25 - 60s - loss: 1.0997 - accuracy: 0.3780 - val_loss: 0.9508 - val_accuracy: 0.5778
Epoch 2/12
25/25 - 59s - loss: 0.8968 - accuracy: 0.6029 - val_loss: 0.5696 - val_accuracy: 0.7167
Epoch 3/12
25/25 - 59s - loss: 0.4877 - accuracy: 0.8064 - val_loss: 0.1893 - val_accuracy: 0.9444
Epoch 4/12
25/25 - 63s - loss: 0.3170 - accuracy: 0.8956 - val_loss: 0.2244 - val_accuracy: 0.9500
Epoch 5/12
25/25 - 59s - loss: 0.2428 - accuracy: 0.9200 - val_loss: 0.1806 - val_accuracy: 0.9278
Epoch 6/12
25/25 - 59s - loss: 0.1888 - accuracy: 0.9345 - val_loss: 0.0885 - val_accuracy: 0.9778
Epoch 7/12
25/25 - 59s - loss: 0.1434 - accuracy: 0.9596 - val_loss: 0.0835 - val_accuracy: 0.9778
Epoch 8/12
25/25 - 59s - loss: 0.1311 - accuracy: 0.9604 - val_loss: 0.2228 - val_accuracy: 0.9556
Epoch 9/12
25/25 - 59s - loss: 0.1186 - accuracy: 0.9642 - val_loss: 0.0234 - val_accuracy: 0.9944
Epoch 10/12
25/25 - 60s - loss: 0.0907 - accuracy: 0.9726 - val_loss: 0.0623 - val_accuracy: 0.9833
Epoch 11/

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])

from time import time

start_time = time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2,
        callbacks=[
           EarlyStopping(patience=4, restore_best_weights=True),
           ReduceLROnPlateau(patience=2)
        ]
      )
print("--- Finished in %s seconds ---" % (time() - start_time))

Epoch 1/20
25/25 - 65s - loss: 1.0936 - accuracy: 0.4040 - val_loss: 0.9730 - val_accuracy: 0.4722
Epoch 2/20
25/25 - 65s - loss: 1.0230 - accuracy: 0.4840 - val_loss: 0.8638 - val_accuracy: 0.6222
Epoch 3/20
25/25 - 65s - loss: 0.6393 - accuracy: 0.7500 - val_loss: 0.5746 - val_accuracy: 0.8222
Epoch 4/20
25/25 - 65s - loss: 0.4310 - accuracy: 0.8491 - val_loss: 0.3365 - val_accuracy: 0.8444
Epoch 5/20
25/25 - 65s - loss: 0.3156 - accuracy: 0.8826 - val_loss: 0.3109 - val_accuracy: 0.8833
Epoch 6/20
25/25 - 65s - loss: 0.2551 - accuracy: 0.9108 - val_loss: 0.2045 - val_accuracy: 0.9222
Epoch 7/20
25/25 - 68s - loss: 0.1998 - accuracy: 0.9360 - val_loss: 0.2225 - val_accuracy: 0.9500
Epoch 8/20
25/25 - 65s - loss: 0.1472 - accuracy: 0.9512 - val_loss: 0.1414 - val_accuracy: 0.9611
Epoch 9/20
25/25 - 65s - loss: 0.1470 - accuracy: 0.9497 - val_loss: 0.1385 - val_accuracy: 0.9722
Epoch 10/20
25/25 - 65s - loss: 0.1685 - accuracy: 0.9543 - val_loss: 0.1308 - val_accuracy: 0.9556
Epoch 11/

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])

from time import time

start_time = time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2,
        callbacks=[
           EarlyStopping(patience=4, restore_best_weights=True),
           ReduceLROnPlateau(patience=2)
        ]
      )
print("--- Finished in %s seconds ---" % (time() - start_time))

Epoch 1/20
25/25 - 66s - loss: 1.0710 - accuracy: 0.3986 - val_loss: 0.9783 - val_accuracy: 0.4611
Epoch 2/20
25/25 - 65s - loss: 0.7798 - accuracy: 0.6441 - val_loss: 0.6434 - val_accuracy: 0.7556
Epoch 3/20
25/25 - 66s - loss: 0.5252 - accuracy: 0.7866 - val_loss: 0.3868 - val_accuracy: 0.8500
Epoch 4/20
25/25 - 66s - loss: 0.4447 - accuracy: 0.8323 - val_loss: 0.4355 - val_accuracy: 0.8556
Epoch 5/20
25/25 - 66s - loss: 0.3918 - accuracy: 0.8590 - val_loss: 0.4577 - val_accuracy: 0.8167
Epoch 6/20


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])

from time import time

start_time = time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2,
        callbacks=[
           EarlyStopping(patience=4, restore_best_weights=True),
           ReduceLROnPlateau(patience=2)
        ]
      )
print("--- Finished in %s seconds ---" % (time() - start_time))

Epoch 1/20
25/25 - 65s - loss: 1.1106 - accuracy: 0.3392 - val_loss: 1.0968 - val_accuracy: 0.3167
Epoch 2/20
25/25 - 65s - loss: 1.0096 - accuracy: 0.4604 - val_loss: 0.7014 - val_accuracy: 0.6389
Epoch 3/20
25/25 - 65s - loss: 0.6138 - accuracy: 0.7485 - val_loss: 0.5221 - val_accuracy: 0.7611
Epoch 4/20
25/25 - 65s - loss: 0.4763 - accuracy: 0.8232 - val_loss: 0.3978 - val_accuracy: 0.8500
Epoch 5/20
25/25 - 66s - loss: 0.3869 - accuracy: 0.8567 - val_loss: 0.5060 - val_accuracy: 0.8389
Epoch 6/20
25/25 - 65s - loss: 0.4018 - accuracy: 0.8483 - val_loss: 0.3652 - val_accuracy: 0.8667
Epoch 7/20
25/25 - 65s - loss: 0.4220 - accuracy: 0.8361 - val_loss: 0.3134 - val_accuracy: 0.8833
Epoch 8/20
25/25 - 65s - loss: 0.3012 - accuracy: 0.8902 - val_loss: 0.3489 - val_accuracy: 0.8833
Epoch 9/20
25/25 - 69s - loss: 0.2487 - accuracy: 0.9108 - val_loss: 0.3162 - val_accuracy: 0.9111
Epoch 10/20
25/25 - 65s - loss: 0.1965 - accuracy: 0.9314 - val_loss: 0.1742 - val_accuracy: 0.9389
Epoch 11/

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])

from time import time

start_time = time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time() - start_time))

Epoch 1/20
25/25 - 61s - loss: 1.1057 - accuracy: 0.3857 - val_loss: 1.0379 - val_accuracy: 0.4778
Epoch 2/20
25/25 - 60s - loss: 0.9092 - accuracy: 0.5625 - val_loss: 0.4784 - val_accuracy: 0.8278
Epoch 3/20
25/25 - 61s - loss: 0.5933 - accuracy: 0.7470 - val_loss: 0.4154 - val_accuracy: 0.7722
Epoch 4/20
25/25 - 60s - loss: 0.4447 - accuracy: 0.8209 - val_loss: 0.3415 - val_accuracy: 0.8889
Epoch 5/20
25/25 - 60s - loss: 0.3921 - accuracy: 0.8613 - val_loss: 0.1968 - val_accuracy: 0.9611
Epoch 6/20
25/25 - 60s - loss: 0.3574 - accuracy: 0.8758 - val_loss: 0.1613 - val_accuracy: 0.9611
Epoch 7/20
25/25 - 60s - loss: 0.3267 - accuracy: 0.8941 - val_loss: 0.1358 - val_accuracy: 0.9667
Epoch 8/20
25/25 - 64s - loss: 0.2878 - accuracy: 0.8918 - val_loss: 0.1171 - val_accuracy: 0.9722
Epoch 9/20
25/25 - 60s - loss: 0.2549 - accuracy: 0.9200 - val_loss: 0.1159 - val_accuracy: 0.9611
Epoch 10/20
25/25 - 60s - loss: 0.1758 - accuracy: 0.9436 - val_loss: 0.2959 - val_accuracy: 0.8833
Epoch 11/

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

Epoch 1/20
25/25 - 61s - loss: 1.0623 - accuracy: 0.4329 - val_loss: 0.8907 - val_accuracy: 0.6278
Epoch 2/20
25/25 - 60s - loss: 0.7797 - accuracy: 0.6540 - val_loss: 0.3596 - val_accuracy: 0.8611
Epoch 3/20
25/25 - 60s - loss: 0.5865 - accuracy: 0.7675 - val_loss: 0.5703 - val_accuracy: 0.8333
Epoch 4/20
25/25 - 60s - loss: 0.4738 - accuracy: 0.8148 - val_loss: 0.2539 - val_accuracy: 0.9389
Epoch 5/20
25/25 - 60s - loss: 0.3829 - accuracy: 0.8666 - val_loss: 0.2359 - val_accuracy: 0.9500
Epoch 6/20
25/25 - 60s - loss: 0.3124 - accuracy: 0.8956 - val_loss: 0.0932 - val_accuracy: 0.9889
Epoch 7/20
25/25 - 63s - loss: 0.2023 - accuracy: 0.9306 - val_loss: 0.1108 - val_accuracy: 0.9722
Epoch 8/20
25/25 - 60s - loss: 0.1540 - accuracy: 0.9543 - val_loss: 0.0700 - val_accuracy: 0.9667
Epoch 9/20
25/25 - 60s - loss: 0.1323 - accuracy: 0.9604 - val_loss: 0.1041 - val_accuracy: 0.9667
Epoch 10/20
25/25 - 60s - loss: 0.1338 - accuracy: 0.9573 - val_loss: 0.0460 - val_accuracy: 0.9833
Epoch 11/

Dari sini, didapati akurasi sebesar **99.44%** pada `val` data, sedangkan akurasi sebesar **98.02%** pada `train` data sendiri. Untuk waktu eksekusi, training dan validasi data memakan waktu selama **1411 s** ~ **24 min**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16


prior = VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(150, 150, 3)
)

model = Sequential(name="simple_sequential")
model.add(prior)
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))
# model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True)

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=7,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))